# **Description**:

This script was developed to replicate Tables 7, 8, and 9 on the manuscript "Relationship Between Environmental Values and the Evaluation of Watershed Conservation Plans that Benefit the Community Versus the Individual." It is designed to be executed in R.

For ease of replication, an R Colab version is also available, allowing users to run the analysis in a cloud-based environment using the R kernel.

To prepare the final tables for presentation in the manuscript, the resulting values were formatted in Excel.

------

**Analysis:**

- Correlation between environmental values and participants’ preferences for conservation plans
- Relationship of environmental values and participants’ preferences for conservation plans

**Project:**

Relationship Between Environmental Values and the Evaluation of Watershed Conservation Plans that Benefit the Community Versus the Individual

**Author:**

Efrain Noa Yarasca PhD.

**Input:**

'Rating_of_participants_to_the_suggested_plans.csv'
'EPVQ_score_data.csv'

**Output:**

- Table 7: Correlation Coefficients and p-Values Between the Environmental Values and the Participants Preferences Metric (PPM) for Plans.

- Table 8: Variance Inflation Factors (VIF) for Environmental Human Values in Test- and End-Users.

- Table 9: Regression Weights, t-Values, and p-Values from the Multi-Linear Regression.
# ----------------------------------------------------------------------------#

## Install libraries

Installing libraries may take some minutes

In [5]:
install.packages("dplyr", repos = "http://cran.rstudio.com/")
install.packages("tibble", repos = "http://cran.rstudio.com/")
install.packages("car", repos = "http://cran.rstudio.com/")

library(dplyr)
library(tibble)
library(car)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘cowplot’, ‘Deriv’, ‘microbenchmark’, ‘Rdpack’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode




## R code


In [ ]:
# -----------------------------------------------------------------------------
# Set working directory
setwd("D:/work/research_o/paper_02/replica") # No needed in Colab


## **Part 1: Get PPM values**

In [6]:
# ===================== Part 1: Get PPM values  ====================== #
# 1.1. Read the initial dataset.
SP_df <- read.csv('Rating_of_participants_to_the_suggested_plans.csv')


# 1.2. Reshaping the SP_df before calculations
SP_df_transp <- SP_df %>%
  select(-Particip) %>%
  column_to_rownames("pid") %>%
  t() %>%
  as.data.frame() %>%
  rownames_to_column("pid") %>%
  mutate(ID = row_number(), .before = 1)
# head(SP_df_transp)



# 1.3. Settings and selection of data
start_col_num <- 3  # Column 'p1' is at index 3
end_col_num <- 79   # Column 'p24' is at index 26
column_indices <- start_col_num:end_col_num

average_PPMs_vector <- c() # Set an empty vector to store average PPM values (slopes)
chunk_size <- 4

# 1.4. Compute PPM. Loop through each End-User (from p1 to p24)
for (col_index in column_indices) {
  slopes <- c()
  # Loop through the data in chunks of 4 rows for the current test-user
  for (i in 1:(nrow(SP_df_transp)/chunk_size)) {
    start_row <- (i - 1) * chunk_size + 1
    end_row <- i * chunk_size
    chunk_data <- SP_df_transp[start_row:end_row, ]

    # Compute the PPM value using linear regression approach
    model <- lm(chunk_data[[col_index]] ~ ID, data = chunk_data)
    slopes <- c(slopes, -coef(model)[2])
  }
  average_PPM_for_col <- mean(slopes)
  average_PPMs_vector <- c(average_PPMs_vector, average_PPM_for_col)
}

# 1.5. Print the final vector of average PPM values
# cat("Average PPMs for columns p1 to p24:\n")
# cat(paste(round(average_PPMs_vector, 4), collapse = ", "))

# Create a data frame from the average_PPMs_vector
ppm_df <- as.data.frame(average_PPMs_vector)
colnames(ppm_df) <- "PPM"  # Rename the column to 'PPM'
user_column <- c(rep("end_user", 24), rep("test_user", 53))  # Create the 'user' column
pid_column <- paste0("p", seq_len(length(user_column)))

# Create a data frame
final_df <- data.frame(
  pid = pid_column,
  user = user_column,
  PPM = average_PPMs_vector
)
final_df$PPM <- round(final_df$PPM, 3)  # Round the 'PPM'

# Print the PMs values
#print(final_df)
print(head(final_df))
# ==============================================================================

  pid     user  PPM
1  p1 end_user 0.44
2  p2 end_user 0.56
3  p3 end_user 0.64
4  p4 end_user 0.92
5  p5 end_user 0.48
6  p6 end_user 0.50


## **Part 2: Get EPVQ averages by each Human-value**

In [7]:
# ==============================================================================
# =============== Part 2: Get EPVQ averages by each Human-value ================

# 2.1. Read the initial dataset.
epvq_df <- read.csv('EPVQ_score_data.csv')


# 2.2. Reshaping the epvq_df before calculations
epvq_df_transp <- epvq_df %>%
  select(-Particip) %>%
  column_to_rownames("pid") %>%
  t() %>%
  as.data.frame() %>%
  rownames_to_column("pid") %>%
  mutate(ID = row_number(), .before = 1)
# head(epvq_df_transp)


# 2.3. Settings and selection of data
row_chunks <- list(
  1:4,    # Biospheric
  5:9,    # Altruistic
  10:12,  # Hedonic
  13:17   # Egoistic
)

# Define the range of columns to process (End-users: p1-p24; Test-users: p25-p77)
start_col_name <- "p1" # "p25" #
end_col_name <- "p77" # "p77" #
column_indices <- which(names(epvq_df_transp) == start_col_name):which(names(epvq_df_transp) == end_col_name)
column_names <- names(epvq_df_transp)[column_indices]

# Initialize a data frame to store the average values
average_values_matrix <- matrix(NA, nrow = length(row_chunks), ncol = length(column_names))
average_values_df <- as.data.frame(average_values_matrix)
names(average_values_df) <- column_names

# 2.4. Compute PPM. Loop through each column
for (col_index in 1:length(column_names)) {
  current_col_name <- column_names[col_index]

  # Loop through each Human-value
  for (chunk_index in 1:length(row_chunks)) {
    chunk_rows <- row_chunks[[chunk_index]]
    chunk_data <- epvq_df_transp[chunk_rows, current_col_name]
    average_value <- mean(chunk_data)
    average_values_df[chunk_index, current_col_name] <- average_value
  }
}
# Print average E-PVQ values
print(round(average_values_df, 2))
# ==============================================================================

    p1   p2   p3   p4   p5   p6   p7   p8   p9  p10  p11  p12  p13  p14 p15 p16
1 5.25 4.75 6.75 6.25 6.50 5.75 6.00 5.75 6.25 5.50 6.50 5.50 6.25 6.75 4.5 3.5
2 6.80 7.00 6.60 6.60 7.00 6.40 6.60 6.40 6.80 5.20 6.80 5.60 6.80 7.00 5.2 6.0
3 6.00 6.33 6.67 5.00 6.33 6.33 5.67 5.33 6.00 4.67 6.33 4.67 4.00 7.00 5.0 7.0
4 3.00 2.40 5.20 4.40 3.40 4.00 5.80 5.20 2.60 4.00 5.80 4.40 3.80 6.60 5.0 3.6
   p17 p18  p19  p20  p21  p22  p23  p24  p25  p26  p27  p28  p29  p30  p31 p32
1 4.25 4.5 6.25 5.25 6.25 5.75 6.25 7.00 6.75 5.75 6.00 7.00 5.50 4.25 5.50 7.0
2 6.60 7.0 7.00 5.60 6.60 7.00 6.60 7.00 6.40 6.60 6.60 6.80 4.80 5.80 6.20 7.0
3 7.00 6.0 7.00 4.67 6.33 6.67 5.00 5.67 6.33 6.33 6.67 5.33 6.33 6.00 5.33 7.0
4 3.40 2.8 3.00 4.40 2.60 2.20 4.80 3.40 3.20 6.20 4.20 1.80 4.60 3.20 2.40 3.6
   p33 p34  p35  p36  p37 p38 p39  p40  p41 p42  p43  p44 p45  p46 p47 p48  p49
1 5.75 4.0 3.75 5.25 4.50 6.5 5.0 4.50 7.00 5.5 7.00 5.75 7.0 6.75 6.5 6.5 6.75
2 6.00 6.0 5.60 5.60 5.60 7.0 6.0 6.20 7

## **Part 3: Combine EPVQ and PPM into one df**

In [8]:
# ================= Part 3: Combine EPVQ and PPM into one df ===================
# Transpose, convert to data frame, and assign column names
avg_df_t <- as.data.frame(t(average_values_df))
colnames(avg_df_t) <- c("Bio", "Alt", "Hed", "Ego")
avg_df_t$pid <- rownames(avg_df_t) # Add 'pid' from row names
avg_df_t$user <- c(rep("end_user", 24), rep("test_user", 53)) # Add 'user' column
avg_df_t$PPM <- average_PPMs_vector # Add the PPM column
avg_df_t$Hed <- round(avg_df_t$Hed, 2) # Round 'Hed' values
avg_df_t <- avg_df_t[, c("pid", "user", "Bio", "Alt", "Hed", "Ego", "PPM")] # Reorder columns

# Print result
#print(avg_df_t)
print(head(avg_df_t))
# ==============================================================================

   pid     user  Bio Alt  Hed Ego  PPM
p1  p1 end_user 5.25 6.8 6.00 3.0 0.44
p2  p2 end_user 4.75 7.0 6.33 2.4 0.56
p3  p3 end_user 6.75 6.6 6.67 5.2 0.64
p4  p4 end_user 6.25 6.6 5.00 4.4 0.92
p5  p5 end_user 6.50 7.0 6.33 3.4 0.48
p6  p6 end_user 5.75 6.4 6.33 4.0 0.50


## **Part 4. Get Table 7**

In [9]:
# =========================== Part 4. Get Table 7 ==============================
# Table 7: Correlation Coef and p-Values Between Environmental Values and Participants Preferences Metric (PPM)

# ------------------ For Test-users --------------------
test_user_data <- avg_df_t %>%
  filter(user == "test_user") %>%
  mutate(across(c(PPM, Bio, Alt, Hed, Ego), as.numeric))

human_vals <- c("Bio", "Alt", "Hed", "Ego")
# Initialize list to store results
results_list <- lapply(human_vals, function(feature) {
  pearson <- cor.test(test_user_data$PPM, test_user_data[[feature]], method = "pearson")
  spearman <- cor.test(test_user_data$PPM, test_user_data[[feature]], method = "spearman")

  data.frame(
    Human_value = feature,
    `p-value` = pearson$p.value,
    pearson = pearson$estimate,
    Spearman = spearman$estimate,
    Confid_interval = paste0("(", round(pearson$conf.int[1], 3), ", ", round(pearson$conf.int[2], 3), ")"),
    stringsAsFactors = FALSE
  )
})

# Combine all into a single df and print results
results_df <- do.call(rbind, results_list)
print(results_df, row.names = FALSE)  # Print Table 7 for Test-users


# ---------------------- For End-users ----------------------
end_user_data <- avg_df_t %>%
  filter(user == "end_user") %>%
  mutate(across(c(PPM, Bio, Alt, Hed, Ego), as.numeric))

human_vals <- c("Bio", "Alt", "Hed", "Ego")
# Initialize list to store results
results_list <- lapply(human_vals, function(feature) {
  pearson <- cor.test(end_user_data$PPM, end_user_data[[feature]], method = "pearson")
  spearman <- cor.test(end_user_data$PPM, end_user_data[[feature]], method = "spearman")

  data.frame(
    Human_value = feature,
    `p-value` = pearson$p.value,
    pearson = pearson$estimate,
    Spearman = spearman$estimate,
    Confid_interval = paste0("(", round(pearson$conf.int[1], 3), ", ", round(pearson$conf.int[2], 3), ")"),
    stringsAsFactors = FALSE
  )
})

# Combine all into a single df and print results
results_df <- do.call(rbind, results_list)
print(results_df, row.names = FALSE) # Print Table 7 for End-users
# ==============================================================================

Warning message in cor.test.default(test_user_data$PPM, test_user_data[[feature]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(test_user_data$PPM, test_user_data[[feature]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(test_user_data$PPM, test_user_data[[feature]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(test_user_data$PPM, test_user_data[[feature]], :
“Cannot compute exact p-value with ties”


 Human_value      p.value     pearson    Spearman  Confid_interval
         Bio 1.073426e-02 -0.34771978 -0.27381577 (-0.565, -0.085)
         Alt 3.785996e-05 -0.53423643 -0.43025563 (-0.703, -0.308)
         Hed 5.590002e-01 -0.08208522 -0.04049995  (-0.345, 0.192)
         Ego 4.312456e-04  0.46641701  0.52740521   (0.224, 0.654)


Warning message in cor.test.default(end_user_data$PPM, end_user_data[[feature]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(end_user_data$PPM, end_user_data[[feature]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(end_user_data$PPM, end_user_data[[feature]], :
“Cannot compute exact p-value with ties”
Warning message in cor.test.default(end_user_data$PPM, end_user_data[[feature]], :
“Cannot compute exact p-value with ties”


 Human_value      p.value    pearson   Spearman  Confid_interval
         Bio 6.932064e-02 -0.3770625 -0.3019452  (-0.677, 0.031)
         Alt 5.225873e-05 -0.7295534 -0.6245490 (-0.875, -0.462)
         Hed 1.226159e-01 -0.3238701 -0.2318939  (-0.643, 0.091)
         Ego 1.744017e-02  0.4806191  0.5938948    (0.096, 0.74)


## **Part 5: Get Table 8**

In [10]:
# =========================== Part 5: Get Table 8 =============================
# Table 8: Variance Inflation Factors (VIF) for Env. Human Values in Test- and End-Users
# install.packages("car", repos = "http://cran.rstudio.com/")
# library(car)

# -------------------- For Test-user
LR_test_user <- lm(PPM ~ Bio + Alt + Hed + Ego, data=test_user_data)
vif_test_user <- vif(LR_test_user)   # Compute VIF
print(vif_test_user)    # Print VIF values

# -------------------- For End-user
LR_end_user <- lm(PPM ~ Bio + Alt + Hed + Ego, data=end_user_data)
vif_end_user <- vif(LR_end_user)   # Compute VIF
print(vif_end_user)    # Print VIF values
# ==============================================================================

     Bio      Alt      Hed      Ego 
2.153851 3.265374 2.068175 1.244434 
     Bio      Alt      Hed      Ego 
1.762713 2.298034 1.595638 1.355064 


## **Part 6: Get Table 9**

In [11]:
# ==============================================================================
# ======================= Part 6: Get Table 9 ============================
# Table 9: Regression Weights, t-Values, and p-Values from the Regression Between
#          the Environmental Values and the Participant Preference Metric (PPM)

# -------------------- For Test-user
LR_test_user <- lm(PPM ~ Bio + Alt + Hed + Ego, data=test_user_data)
# summary(LR_test_user)
print(summary(LR_test_user)$coefficients)


# -------------------- For End-user
LR_end_user <- lm(PPM ~ Bio + Alt + Hed + Ego, data=end_user_data)
# summary(LR_test_user)
print(summary(LR_end_user)$coefficients)
# ==============================================================================

                Estimate Std. Error    t value     Pr(>|t|)
(Intercept)  1.457127025 0.29486706  4.9416405 9.831611e-06
Bio          0.005651481 0.04691175  0.1204705 9.046133e-01
Alt         -0.307231818 0.07810917 -3.9333642 2.689835e-04
Hed          0.082235113 0.05454744  1.5075888 1.382138e-01
Ego          0.129460101 0.03377673  3.8328188 3.682868e-04
                Estimate Std. Error     t value     Pr(>|t|)
(Intercept)  2.754867136 0.55112802  4.99859752 7.974911e-05
Bio         -0.106173292 0.06164423 -1.72235559 1.012442e-01
Alt         -0.271467436 0.11036662 -2.45968777 2.366345e-02
Hed         -0.005272481 0.05919541 -0.08906908 9.299592e-01
Ego          0.116287751 0.04043406  2.87598501 9.675539e-03
